In [17]:
import json
import requests
import pandas as pd

def makeUrlParam(url, param):
    params = ""
    for key, val in param.items():
        params += "&" + key + "=" + val
    url = url + "?" + params[1:]
    return url
res_dic = []
query_subject = "게임개발"
query_date = [
    { "start_date" : "20190101", "end_date" : "20190630" },
    { "start_date" : "20190701", "end_date" : "20191231" },
    { "start_date" : "20200101", "end_date" : "20200630" },
    { "start_date" : "20200701", "end_date" : "20201231" },
    { "start_date" : "20210101", "end_date" : "20210531" }
]
stop_words = ["임차", "개최", "축제", "페스티발", "페스티벌", "조사", "발간", "연구", "홍보", "인력양성", "위탁관리", "위탁처리", "자격검정", "폐기", "완구", "컨설팅", "수출", "양성"]
url = "http://apis.data.go.kr/1230000/BidPublicInfoService02/getBidPblancListInfoServcPPSSrch"
param = {
    "numOfRows" : "500",
    "pageNo" : "1",
    "ServiceKey" : "AiMHAf%2BJJUylxroBA5LeiG%2FkXWULTMXWujS9xTtiD0TO23mWyND4uvsmnWlTJEZcr%2FWq5q6qzQBi0Ps2KrlD%2Fg%3D%3D",
    "inqryDiv" : "1", #검색하고자하는 조회구분 1.등록일시, 2.입찰공고번호 3.변경일시
    "type" : "json"
}
param["bidNtceNm"] = query_subject

In [18]:
for query in query_date:
    print("데이터 로드중... query_date : {} ~ {}".format(query["start_date"], query["end_date"]))
    param["inqryBgnDt"] = query["start_date"] + "0000"
    param["inqryEndDt"] = query["end_date"] + "0000"
    request_url = makeUrlParam(url, param)
    print("request_url : {}".format(request_url))
    res = requests.get(request_url)
    try:
        res_arr = json.loads(res.text)
        if res_arr["response"]["header"]["resultCode"] == "00":
            for item in res_arr["response"]["body"]["items"]:
                stop_word_check = False
                for stop_word in stop_words:
                    if stop_word in item["bidNtceNm"]:
                        stop_word_check = True
                        break
                if stop_word_check == False:
#                     print(item["bidNtceNm"])
                    this_dic = {}
                    this_dic["게시일"] = item["bidNtceDt"]
                    this_dic["용역명"] = item["bidNtceNm"]
                    this_dic["기초금액"] = item["asignBdgtAmt"]
                    this_dic["공고문"] = '=HYPERLINK("{}", "{}")'.format(item["ntceSpecDocUrl1"], "링크")
                    this_dic["제안요청서"] = '=HYPERLINK("{}", "{}")'.format(item["ntceSpecDocUrl2"], "링크")
                    this_dic["기타"] = '=HYPERLINK("{}", "{}")'.format(item["ntceSpecDocUrl3"], "링크")
                    res_dic.append(this_dic)
            print("데이터 로드완료. query_date : {} ~ {}".format(query["start_date"], query["end_date"]))
            print("--------------------------------------------------------")
        else:
            print("query_date : {} ~ {}, resultMsg : {}".format(query["start_date"], query["end_date"], res_arr["response"]["header"]["resultMsg"]))
            print("--------------------------------------------------------")
    except:
        print("json load 오류발생 : query_date : {} ~ {}".format(query["start_date"], query["end_date"]))
        print(res.text)
        print("--------------------------------------------------------")
        break
        
df = pd.DataFrame(res_dic)
df.to_excel("나라장터/output.xlsx", index=False)
print("총 {}건 로드완료".format(len(res_dic)))

데이터 로드중... query_date : 20190101 ~ 20190630
request_url : http://apis.data.go.kr/1230000/BidPublicInfoService02/getBidPblancListInfoServcPPSSrch?numOfRows=500&pageNo=1&ServiceKey=AiMHAf%2BJJUylxroBA5LeiG%2FkXWULTMXWujS9xTtiD0TO23mWyND4uvsmnWlTJEZcr%2FWq5q6qzQBi0Ps2KrlD%2Fg%3D%3D&inqryDiv=1&type=json&bidNtceNm=게임개발&inqryBgnDt=201901010000&inqryEndDt=201906300000
데이터 로드완료. query_date : 20190101 ~ 20190630
--------------------------------------------------------
데이터 로드중... query_date : 20190701 ~ 20191231
request_url : http://apis.data.go.kr/1230000/BidPublicInfoService02/getBidPblancListInfoServcPPSSrch?numOfRows=500&pageNo=1&ServiceKey=AiMHAf%2BJJUylxroBA5LeiG%2FkXWULTMXWujS9xTtiD0TO23mWyND4uvsmnWlTJEZcr%2FWq5q6qzQBi0Ps2KrlD%2Fg%3D%3D&inqryDiv=1&type=json&bidNtceNm=게임개발&inqryBgnDt=201907010000&inqryEndDt=201912310000
데이터 로드완료. query_date : 20190701 ~ 20191231
--------------------------------------------------------
데이터 로드중... query_date : 20200101 ~ 20200630
request_url : http://apis.da

In [19]:
res_dic

[{'게시일': '2019-06-24 20:41:06',
  '용역명': '2019 초등 진로활동 보드게임 개발',
  '기초금액': '49355000',
  '공고문': '=HYPERLINK("http://www.g2b.go.kr:8081/ep/co/fileDownload.do?fileTask=NOTIFY&fileSeq=20190634348::00::1::1", "링크")',
  '제안요청서': '=HYPERLINK("http://www.g2b.go.kr:8081/ep/co/fileDownload.do?fileTask=NOTIFY&fileSeq=20190634348::00::2::2", "링크")',
  '기타': '=HYPERLINK("", "링크")'},
 {'게시일': '2019-06-28 18:49:59',
  '용역명': '2019 초등 진로활동 보드게임 개발',
  '기초금액': '49355000',
  '공고문': '=HYPERLINK("http://www.g2b.go.kr:8081/ep/co/fileDownload.do?fileTask=NOTIFY&fileSeq=20190640958::00::1::1", "링크")',
  '제안요청서': '=HYPERLINK("http://www.g2b.go.kr:8081/ep/co/fileDownload.do?fileTask=NOTIFY&fileSeq=20190640958::00::2::2", "링크")',
  '기타': '=HYPERLINK("", "링크")'},
 {'게시일': '2019-07-05 17:22:05',
  '용역명': '2019 초등 진로활동 보드게임 개발',
  '기초금액': '49355000',
  '공고문': '=HYPERLINK("http://www.g2b.go.kr:8081/ep/co/fileDownload.do?fileTask=NOTIFY&fileSeq=20190709200::00::1::1", "링크")',
  '제안요청서': '=HYPERLINK("http://www.g2b.